In [37]:
# Imports
import gurobipy
import colorama

In [38]:
class Model:
  # Flag indicating whether to check the validity and consistency of constructor arguments
  __VALIDATE = True

  # Tutoring modes
  IN_PERSON = 0
  VIRTUAL = 1

  # Availability/day preference codes
  UNAVAILABLE = 0
  NOT_PREFERRED = 0.5
  PREFERRED = 1

  # Mode preference codes
  PREFER_VIRTUAL = -1
  NO_PREFERENCE = 0
  PREFER_IN_PERSON = 1

  # Status codes
  LOADING = 0     # The model is being initialized
  OPTIMAL = 1     # An optimal solution has been found
  INFEASIBLE = 2  # The model is infeasible

  """
  Arguments:
    * N_T: the number of tutors
    * N_D: the number of days
    * lp,lv: the minimum number of tutors needed in-person/virtually each day
    * s: the minimum number of shifts a tutor needs to work
    * sp: the minimum number of in-person shifts a tutor needs to work
    * S: S[t] = the maximum number of shifts tutor t is willing to work
    * a: a[t][d] = the availability of tutor t on day d
    * m: m[t] = tutor t's preferred mode (or lack thereof)
    * fp: fp[d] = the target fraction of total shifts that are in-person on day d
    * fv: fv[d] = the target fraction of total shifts that are virtual on day d
    * w1,w2,w3,w4: the weights of the objective function
  """
  def __init__(self, N_T:int, N_D:int, lp:int, lv:int, S:tuple[int], s:int, sp:int, a:tuple[tuple[int]],
               m:tuple[int],fp:tuple[float],fv:tuple[float], w1:float, w2:float, w3:float, w4:float):
    self.model = gurobipy.Model("")
    self.status = Model.LOADING
    self.N_T, self.N_D = N_T, N_D
    self.__set_decision_variables(N_T,N_D)
    self.__set_constraints(lp,lv,s,sp,S,a)
    self.__set_objective_function(a,m,fp,fv,w1,w2,w3,w4)
    self.__solve()

  """
  A helper function that either (1) applies f to x if x is not a list or (2) applies f to any
  non-list elements in x if x is a list
  """
  def __unwrap_then_map(x,f):
    if isinstance(x,list):
      return [Model.__unwrap_then_map(x_,f) for x_ in x]
    else:
      return f(x)

  """
  Arguments: idx = (mode, tutors, days) for
    * mode: the tutoring mode (Model.IN_PERSON or Model.VIRTUAL)
    * tutors: the tutor(s)
    * days: the day(s)
  Returns:
    * None if the model is INFEASIBLE
    * An element/array/matrix of decision variables if the model is LOADING or the integer values
      of the decision variables if the model is OPTIMAL. For example, if the model has been solved
      to optimality, self[Model.IN_PERSON,:,:] returns a matrix A of indicators such that A_td = 1
      if and only if tutor t is working day d in-person.
  """
  def __getitem__(self,idx):
    mode,tutors,days = idx
    assert mode==Model.IN_PERSON or mode==Model.VIRTUAL
    if self.status==Model.INFEASIBLE:
      return None
    
    rows = self.__xp[tutors] if mode==Model.IN_PERSON else self.__xv[tutors]
    if len(rows)==0:
      return []
    elif not isinstance(rows[0],list):
      return rows[days] if self.status==Model.LOADING else Model.__unwrap_then_map(rows[days],lambda x: int(x.X))
    return [row[days] for row in rows] if self.status==Model.LOADING else Model.__unwrap_then_map([row[days] for row in rows],lambda x: int(x.X))

  def __set_decision_variables(self,N_T,N_D):
    if Model.__VALIDATE:
      assert N_T>0
      assert N_D>0
    self.__xp = [[self.model.addVar(vtype=gurobipy.GRB.BINARY,name=f"xp_({t},{d})") for d in range(N_D)] for t in range(N_T)]
    self.__xv = [[self.model.addVar(vtype=gurobipy.GRB.BINARY,name=f"xv_({t},{d})") for d in range(N_D)] for t in range(N_T)]

  def __set_constraints(self,lp,lv,s,sp,S,a):
    N_T, N_D = self.N_T, self.N_D
    if Model.__VALIDATE:
      assert lp>=0
      assert lv>=0
      assert s>=0
      assert sp>=0
      assert len(S)==N_T                      
      assert sum([S_t>=0 for S_t in S])==N_T
      assert len(a)==N_T
      assert sum([len(row)==N_D for row in a])==N_T
      assert sum([a[t][d] in (Model.UNAVAILABLE,Model.NOT_PREFERRED,Model.PREFERRED)for t in range(N_T) for d in range(N_D)])==N_T*N_D

    for t in range(N_T):
      # the number of in-person/virtual shifts scheduled for tutor t
      Xp_t, Xv_t = sum(self[Model.IN_PERSON,t,:]), sum(self[Model.VIRTUAL,t,:])
      self.model.addConstr(Xp_t+Xv_t<=S[t])  # t cannot be scheduled for more than S[t] shifts
      self.model.addConstr(Xp_t+Xv_t>=s)     # t must work at least s shifts total
      self.model.addConstr(Xp_t>=sp)         # t must work at least sp in-person shifts
      for d in range(N_D):
        if a[t][d]==Model.UNAVAILABLE:
          # tutors cannot be scheduled on days they're unavailable
          self.model.addConstr(self[Model.IN_PERSON,t,d]==0)
          self.model.addConstr(self[Model.VIRTUAL,t,d]==0)
        else:
          # tutors cannot work both an in-person and virtual shift on the same day
          self.model.addConstr(self[Model.IN_PERSON,t,d]+self[Model.VIRTUAL,t,d]<=1)
    for d in range(N_D):
      # there must be at least lp/lv tutors working in-person/virtual each day
      self.model.addConstr(sum(self[Model.IN_PERSON,:,d])>=lp)
      self.model.addConstr(sum(self[Model.VIRTUAL,:,d])>=lv)

  def __set_objective_function(self,a,m,fp,fv,w1,w2,w3,w4):
    epsilon = 10**-3
    N_T, N_D = self.N_T, self.N_D
    if Model.__VALIDATE:
      assert len(m)==N_T
      assert sum([m_t in (Model.PREFER_VIRTUAL,Model.NO_PREFERENCE,Model.PREFER_IN_PERSON) for m_t in m])==N_T
      assert len(fp)==N_D
      assert sum([0<=f for f in fp])==N_D
      assert len(fv)==N_D
      assert sum([0<=f for f in fv])==N_D
      assert abs(1-sum(fp)-sum(fv))<epsilon  # the target fractions should add to 1
      assert w1>=0
      assert w2>=0
      assert w3>=0
      assert w4>=0

    x = [[self[Model.IN_PERSON,t,d]+self[Model.VIRTUAL,t,d] for d in range(N_D)] for t in range(N_T)]
    X = sum([x[t][d] for t in range(N_T) for d in range(N_D)])
    ALIGN = -1/N_T*sum([(X*fp[d]-sum(self[Model.IN_PERSON,:,d]))**2+(X*fv[d]-sum(self[Model.VIRTUAL,:,d]))**2 for d in range(N_D)])
    DAY_PREF = -3*X+4*sum([a[t][d]*x[t][d] for t in range(N_T) for d in range(N_D)])
    MODE_PREF = sum([(self.__xp[t][d]-self.__xv[t][d])*m[t] for t in range(N_T) for d in range(N_D)])
    self.model.setObjective(w1*X+w2*ALIGN+w3*DAY_PREF+w4*MODE_PREF,gurobipy.GRB.MAXIMIZE)

  def __solve(self):
    self.model.setParam("OutputFlag",False)
    self.model.optimize()
    self.status = Model.OPTIMAL if self.model.status==gurobipy.GRB.OPTIMAL else Model.INFEASIBLE

In [39]:
class Schedule:
  # Availability/day preference codes
  UNAVAILABLE = Model.UNAVAILABLE
  NOT_PREFERRED = Model.NOT_PREFERRED
  PREFERRED = Model.PREFERRED

  # Mode preference codes
  PREFER_VIRTUAL = Model.PREFER_VIRTUAL
  NO_PREFERENCE = Model.NO_PREFERENCE
  PREFER_IN_PERSON = Model.PREFER_IN_PERSON

  # Tutoring modes
  IN_PERSON = Model.IN_PERSON
  VIRTUAL = Model.VIRTUAL
  EITHER = max(Model.IN_PERSON,Model.VIRTUAL)+1

  """
  Arguments:
    * tutors: the tutors to schedule
    * days: the days to schedule
    * min_tutors_ip,min_tutors_v: the minimum number of tutors needed in-person/virtually each day
    * min_tutor_shifts: the minimum number of shifts a tutor needs to work
    * min_tutor_ip_shifts: the minimum number of in-person shifts a tutor needs to work
    * max_shifts_by_tutor: max_shifts_by_tutor[t] = the maximum number of shifts tutor t is willing to work
    * mode_preference: mode_preference[t] = tutor t's preferred mode (or lack thereof)
    * availability: a[t][d] = the availability of tutor t on day d
    * target_ip_fractions: target_ip_fractions[d] = the target fraction of total shifts that are in-person on day d
    * target_v_fractions: target_v_fractions[d] = the target fraction of total shifts that are virtual on day d
    * SHIFTS_weight: the emphasis placed on the number of shifts scheduled
    * ALIGN_weight: the emphasis placed on aligning shift scheduling with the target fractions
    * DAY_PREF_weight: the emphasis placed on matching tutor work day preferences
    * MODE_PREF_weight: the emphasis placed on matching tutor work mode preferences
  """
  def __init__(self,tutors,days,min_tutors_ip,min_tutors_v,min_tutor_shifts,min_tutor_ip_shifts,
              max_shifts_by_tutor,mode_preference,availability,target_ip_fractions,target_v_fractions,
              SHIFTS_weight,ALIGN_weight,DAY_PREF_weight,MODE_PREF_weight):
    self.tutors = [tutor for tutor in tutors]
    self.__tutor_index = {tutor:self.tutors.index(tutor) for tutor in self.tutors}
    assert len(self.__tutor_index.keys())==len(self.tutors)  # no repeated tutors

    self.days = [day for day in days]
    self.__day_index = {day:self.days.index(day) for day in self.days}
    assert len(self.__day_index.keys())==len(self.days)  # no repeated days

    self.__model = Model(
      N_T=len(self.tutors),
      N_D=len(self.days),
      lp=min_tutors_ip,
      lv=min_tutors_v,
      s=min_tutor_shifts,
      sp=min_tutor_ip_shifts,
      S=[max_shifts_by_tutor[tutor] for tutor in self.tutors],
      m=[mode_preference[tutor] for tutor in self.tutors],
      a=[[availability[tutor][day] for day in self.days] for tutor in self.tutors],
      fp=[target_ip_fractions[day] for day in self.days],
      fv=[target_v_fractions[day] for day in self.days],
      w1=SHIFTS_weight,
      w2=ALIGN_weight,
      w3=DAY_PREF_weight,
      w4=MODE_PREF_weight
    )

  # Returns True if an optimal schedule has been found and False otherwise
  def schedule_found(self):
    return self.__model.status==Model.OPTIMAL

  def __validate_schedule(self):
    if not self.schedule_found():
      raise Exception("Schedule not found")

  def __validate_mode(mode):
    if mode!=Schedule.IN_PERSON and mode!=Schedule.VIRTUAL and mode!=Schedule.EITHER:
      raise Exception("The provided mode is not one of Schedule.IN_PERSON, Schedule.VIRTUAL, or Schedule.EITHER")
    
  def __validate_tutor(self,tutor):
    if tutor not in self.tutors:
      raise Exception(f"\"{tutor}\" is not a recognized tutor")
  
  def __validate_day(self,day):
    if day not in self.days:
      raise Exception(f"\"{day}\" is not a recognized day")

  # Returns True if the given tutor is scheduled to work the given day via the given mode and False otherwise
  def check(self,mode,tutor,day):
    Schedule.__validate_mode(mode)
    self.__validate_tutor(tutor)
    self.__validate_day(day)
    self.__validate_schedule()
    t,d = self.__tutor_index[tutor], self.__day_index[day]
    if mode==Schedule.IN_PERSON:
      return self.__model[Model.IN_PERSON,t,d]==1
    elif mode==Schedule.VIRTUAL:
      return self.__model[Model.VIRTUAL,t,d]==1
    else:
      return self.__model[Model.IN_PERSON,t,d]==1 or self.__model[Model.VIRTUAL,t,d]==1

  # Returns an array containing the days that the given tutor works via the given mode
  def get_tutor_schedule(self,mode,tutor):
    Schedule.__validate_mode(mode)
    self.__validate_tutor(tutor)
    self.__validate_schedule()
    
    t = self.__tutor_index[tutor]
    if mode==Schedule.IN_PERSON:
      return [self.days[d] for d in range(len(self.days)) if self.__model[Model.IN_PERSON,t,d]==1]
    elif mode==Schedule.VIRTUAL:
      return [self.days[d] for d in range(len(self.days)) if self.__model[Model.VIRTUAL,t,d]==1]
    else:
      return [self.days[d] for d in range(len(self.days)) if self.__model[Model.IN_PERSON,t,d]==1 or self.__model[Model.VIRTUAL,t,d]==1]
  
  # Returns an array containing the tutors that work the given day via the given mode
  def get_day_schedule(self,mode,day):
    Schedule.__validate_mode(mode)
    self.__validate_day(day)
    self.__validate_schedule()
    
    d = self.__day_index[day]
    if mode==Schedule.IN_PERSON:
      return [self.tutors[t] for t in range(len(self.tutors)) if self.__model[Model.IN_PERSON,t,d]==1]
    elif mode==Schedule.VIRTUAL:
      return [self.tutors[t] for t in range(len(self.tutors)) if self.__model[Model.VIRTUAL,t,d]==1]
    else:
      return [self.tutors[t] for t in range(len(self.tutors)) if self.__model[Model.IN_PERSON,t,d]==1 or self.__model[Model.VIRTUAL,t,d]==1]
  
  # Prints the schedule in tabular form
  def print_schedule(self):
    if not self.schedule_found():
      print("No schedule found")
    else:
      print(f"{colorama.Fore.BLUE}P{colorama.Style.RESET_ALL} = scheduled for in-person")
      print(f"{colorama.Fore.MAGENTA}V{colorama.Style.RESET_ALL} = scheduled for virtual")
      print(f"{colorama.Fore.RED}X{colorama.Style.RESET_ALL} = not scheduled\n")

      padding = "  "
      t_format = "{:>"+str(max([len(tutor) for tutor in self.tutors]))+"}"
      d_format = padding + "{:>"+str(max([len(day) for day in self.days]))+"}"
      e_format = d_format

      column_header = t_format.format("")
      for day in self.days:
        column_header += d_format.format(day)
      column_header += padding + d_format.format("#P") + d_format.format("#V")
      print(column_header)
      
      for tutor in self.tutors:
        t = self.__tutor_index[tutor]
        row = t_format.format(tutor)
        for day in self.days:
          d = self.__day_index[day]
          if self.__model[Model.IN_PERSON,t,d]==1:
            row += e_format.format("P").replace("P",str(colorama.Fore.BLUE)+"P"+str(colorama.Style.RESET_ALL))
          elif self.__model[Model.VIRTUAL,t,d]==1:
            row += e_format.format("V").replace("V",str(colorama.Fore.MAGENTA)+"V"+str(colorama.Style.RESET_ALL))
          else:
            row += e_format.format("X").replace("X",str(colorama.Fore.RED)+"X"+str(colorama.Style.RESET_ALL))
        row += padding +\
            e_format.format(str(sum(self.__model[Model.IN_PERSON,t,:]))) +\
            e_format.format(str(sum(self.__model[Model.VIRTUAL,t,:])))
        print(row)

      print()
      day_ip_counts = t_format.format("#P")
      for day in self.days:
        day_ip_counts += e_format.format(str(sum(self.__model[Model.IN_PERSON,:,self.__day_index[day]])))
      print(day_ip_counts)
      day_v_counts = t_format.format("#V")
      for day in self.days:
        day_v_counts += e_format.format(str(sum(self.__model[Model.VIRTUAL,:,self.__day_index[day]])))
      print(day_v_counts)

In [40]:
# Small Test
U,NP,P = Schedule.UNAVAILABLE,Schedule.NOT_PREFERRED,Schedule.PREFERRED

tutors = ["Adem","Bebby","Chel"]
days = ["Sun","Mon"]
min_tutors_ip = 0
min_tutors_v = 0
min_tutor_shifts = 0
min_tutor_ip_shifts = 0
max_shifts_by_tutor = {
  "Adem": 2,
  "Bebby": 2,
  "Chel": 1
}
mode_preference = {
  "Adem": Schedule.PREFER_VIRTUAL,
  "Bebby": Schedule.PREFER_VIRTUAL,
  "Chel": Schedule.NO_PREFERENCE
}
availability = {
  "Adem": {"Sun":NP,"Mon":P},
  "Bebby": {"Sun":NP,"Mon":P},
  "Chel": {"Sun":NP,"Mon":P}
}
target_ip_fractions = {
  "Sun": 0.5,
  "Mon": 0.5
}
target_v_fractions = {
  "Sun": 0,
  "Mon": 0
}
SHIFTS_weight = 1
ALIGN_weight = 1
DAY_PREF_weight = 1
MODE_PREF_weight = 100
schedule = Schedule(
  tutors,
  days,
  min_tutors_ip,
  min_tutors_v,
  min_tutor_shifts,
  min_tutor_ip_shifts,
  max_shifts_by_tutor,
  mode_preference,
  availability,
  target_ip_fractions,
  target_v_fractions,
  SHIFTS_weight,
  ALIGN_weight,
  DAY_PREF_weight,
  MODE_PREF_weight
)
schedule.print_schedule()

P = scheduled for in-person
V = scheduled for virtual
X = not scheduled

       Sun  Mon     #P   #V
 Adem    V    V      0    2
Bebby    V    V      0    2
 Chel    X    P      1    0

   #P    0    1
   #V    2    2


In [41]:
# Big Test
"""
Arnold: 2, (Sun,P), (Mon,P), (Tue,N), (Wed,U), (Thu,N)
Bob: 2, (Sun,P), (Mon,N), (Tue,U), (Wed,P), (Thu,N)
Charles: 2, (Sun,N), (Mon,P), (Tue,P), (Wed,U), (Thu,U)
Diana: 3, (Sun,N), (Mon,U), (Tue,N), (Wed,P), (Thu,P)
Evelyn: 2, (Sun,U), (Mon,P), (Tue,N), (Wed,P), (Thu,U)
Frederick: 3, (Sun,N), (Mon,N), (Tue,P), (Wed,P), (Thu,U)
Geovanni: 2, (Sun,N), (Mon,N), (Tue,N), (Wed,N), (Thu,N)
Henrietta: 2, (Sun,N), (Mon,N), (Tue,P), (Wed,U), (Thu,P)
Isaac: 2, (Sun,P), (Mon,U), (Tue,P), (Wed,U), (Thu,P)
Juliet: 2, (Sun,P), (Mon,P), (Tue,N), (Wed,U), (Thu,N)
Karen: 2, (Sun,U), (Mon,P), (Tue,P), (Wed,U), (Thu,U)
Lissandra: 2, (Sun,P), (Mon,N), (Tue,P), (Wed,N), (Thu,P)
Maxine: 2, (Sun,N), (Mon,N), (Tue,U), (Wed,N), (Thu,P)
Norman F.R.: 2, (Sun,U), (Mon,U), (Tue,P), (Wed,P), (Thu,N)
Ollie: 2, (Sun,P), (Mon,N), (Tue,U), (Wed,P), (Thu,N)
"""

U,NP,P = Schedule.UNAVAILABLE,Schedule.NOT_PREFERRED,Schedule.PREFERRED

tutors = ["Arnold", "Bob", "Charles", "Diana", "Evelyn", "Frederick", "Geovanni", "Henrietta", "Isaac", 
          "Juliet", "Karen", "Lissandra", "Maxine", "Norman F.R.", "Ollie"]
days = ["Sun","Mon", "Tue", "Wed", "Thu"]
min_tutors_ip = 2
min_tutors_v = 2
min_tutor_shifts = 2
min_tutor_ip_shifts = 1
max_shifts_by_tutor = {
    "Arnold":2, 
    "Bob":2, 
    "Charles":2, 
    "Diana":3, 
    "Evelyn":2, 
    "Frederick":3, 
    "Geovanni":2, 
    "Henrietta":2, 
    "Isaac":2,
    "Juliet":2,
    "Karen":2, 
    "Lissandra":2,
    "Maxine":2,
    "Norman F.R.":2,
    "Ollie":2
}
mode_preferences = {
    "Arnold": Schedule.PREFER_IN_PERSON, 
    "Bob": Schedule.PREFER_IN_PERSON, 
    "Charles": Schedule.PREFER_IN_PERSON, 
    "Diana": Schedule.PREFER_IN_PERSON, 
    "Evelyn": Schedule.PREFER_IN_PERSON, 
    "Frederick": Schedule.PREFER_VIRTUAL, 
    "Geovanni": Schedule.PREFER_VIRTUAL, 
    "Henrietta": Schedule.PREFER_VIRTUAL, 
    "Isaac": Schedule.PREFER_VIRTUAL,
    "Juliet": Schedule.PREFER_VIRTUAL,
    "Karen": Schedule.NO_PREFERENCE, 
    "Lissandra": Schedule.NO_PREFERENCE,
    "Maxine": Schedule.NO_PREFERENCE,
    "Norman F.R.": Schedule.NO_PREFERENCE,
    "Ollie": Schedule.NO_PREFERENCE
}
availability = {
  "Arnold": {"Sun":P, "Mon":P, "Tue":NP, "Wed":U, "Thu":NP},
  "Bob": {"Sun":P, "Mon":NP, "Tue":U, "Wed":P, "Thu":NP},
  "Charles": {"Sun":NP, "Mon":P, "Tue":P, "Wed":U, "Thu":U},
  "Diana": {"Sun":NP, "Mon":U, "Tue":NP, "Wed":P, "Thu":P},
  "Evelyn": {"Sun":U, "Mon":P, "Tue":NP, "Wed":P, "Thu":U},
  "Frederick": {"Sun":NP, "Mon":NP, "Tue":P, "Wed":P, "Thu":U},
  "Geovanni": {"Sun":NP, "Mon":NP, "Tue":NP, "Wed":NP, "Thu":NP},
  "Henrietta": {"Sun":NP, "Mon":NP, "Tue":P, "Wed":U, "Thu":P},
  "Isaac": {"Sun":P, "Mon":U, "Tue":P, "Wed":U, "Thu":P},
  "Juliet": {"Sun":P, "Mon":P, "Tue":NP, "Wed":U, "Thu":NP},
  "Karen": {"Sun":U, "Mon":P, "Tue":P, "Wed":U, "Thu":U},
  "Lissandra": {"Sun":P, "Mon":NP, "Tue":P, "Wed":NP, "Thu":P},
  "Maxine": {"Sun":NP, "Mon":NP, "Tue":U, "Wed":NP, "Thu":P},
  "Norman F.R.": {"Sun":U, "Mon":U, "Tue":P, "Wed":P, "Thu":NP},
  "Ollie": {"Sun":P, "Mon":NP, "Tue":U, "Wed":P, "Thu":NP}
}
target_ip_fractions = {
  "Sun": 0.05,
  "Mon": 0.1,
  "Tue": 0.2,
  "Wed": 0.2,
  "Thu": 0.1
}
target_v_fractions = {
  "Sun": 0.1,
  "Mon": 0.05,
  "Tue": 0.05,
  "Wed": 0.05,
  "Thu": 0.1
}
SHIFTS_weight = 3
ALIGN_weight = 2
DAY_PREF_weight = 1
MODE_PREF_weight = 1
schedule = Schedule(
  tutors,
  days,
  min_tutors_ip,
  min_tutors_v,
  min_tutor_shifts,
  min_tutor_ip_shifts,
  max_shifts_by_tutor,
  mode_preferences,
  availability,
  target_ip_fractions,
  target_v_fractions,
  SHIFTS_weight,
  ALIGN_weight,
  DAY_PREF_weight,
  MODE_PREF_weight
)
schedule.print_schedule()

P = scheduled for in-person
V = scheduled for virtual
X = not scheduled

             Sun  Mon  Tue  Wed  Thu     #P   #V
     Arnold    P    P    X    X    X      2    0
        Bob    P    X    X    P    X      2    0
    Charles    X    P    P    X    X      2    0
      Diana    X    X    P    P    P      3    0
     Evelyn    X    P    X    P    X      2    0
  Frederick    V    X    P    V    X      1    2
   Geovanni    X    V    X    P    X      1    1
  Henrietta    X    X    V    X    P      1    1
      Isaac    X    X    V    X    P      1    1
     Juliet    V    P    X    X    X      1    1
      Karen    X    V    P    X    X      1    1
  Lissandra    X    X    P    X    V      1    1
     Maxine    X    X    X    P    V      1    1
Norman F.R.    X    X    P    V    X      1    1
      Ollie    V    X    X    P    X      1    1

         #P    2    4    6    6    3
         #V    3    2    2    2    2
